## Agile Hardware Design
***
# Sequential Circuits

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Registers!
* Example sequential blocks
* State machines
* Viewing simulation results

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import treadle._

## Registers

* Explicitly declare a register with `Reg(type)`
  * Contrast from Verilog where registers are synthesized
* A register is simply another block, no special semantics for time
  * Simply need to connect its input and its output
* `clock` and `reset` are implicit
* Not covered today: functionality for using other clocks ([multi-clock](https://www.chisel-lang.org/chisel3/docs/explanations/multi-clock.html) and [asynchronous reset](https://www.chisel-lang.org/chisel3/docs/explanations/reset.html)

<img src="images/reg.svg" alt="register schamic" style="width:100%;" align="center"/>

## Other Flavors of `Reg`

* Helper functions streamline

### [Set Initial Value](https://www.chisel-lang.org/api/latest/chisel3/RegInit$.html) - `RegInit(init)`
* Value applied synchronously when `reset` true

### [Attach Input](https://www.chisel-lang.org/api/latest/chisel3/RegNext$.html) - `RegNext(next)`
* Useful for delaying a signal by a cycle
* Connects input in same line

### [Enable](https://www.chisel-lang.org/api/latest/chisel3/util/RegEnable$.html) - `RegEnable(next, en)`
* Write enable for when to update

In [ ]:
class RegLand extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val en  = Input(Bool())
        val out = Output(Bool())
    })
    val r = Reg(Bool())
//     val r = RegInit(0.B)
    r := io.in
    io.out := r
//     io.out := RegNext(io.in, 0.B)
//     io.out := RegEnable(io.in, 0.B, io.en)
}
println(getVerilog(new RegLand))

## Example: Counter (done manually)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = Reg(UInt(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    val applyEn = Mux(io.en, nextVal, count)
    count := Mux(reset.asBool, 0.U, applyEn)
    io.out := count
}
println(getVerilog(new MyCounter(15)))

## Example: Counter (using RegInit)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    count := Mux(io.en, nextVal, count)
    io.out := count
}
println(getVerilog(new MyCounter(15)))

## Example: Counter (using when)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}
println(getVerilog(new MyCounter(15)))

## Example: Counter (using RegEnable, too dense?)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt(log2Ceil(maxVal+1).W))
    })
    io.count := RegEnable(Mux(io.count < maxVal.U, io.count + 1.U, 0.U), 0.U, io.en)
}
println(getVerilog(new MyCounter(15)))

## Testing MyCounter

In [ ]:
test(new MyCounter(3)) { c =>
// test(new MyCounter(3), Seq(WriteVcdAnnotation)) { c =>
    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(1.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(2.U)
    c.clock.step()

    c.io.en.poke(0.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()
}

## Chisel `Enum`

* `Enum` is Chisel object that assigns `UInt`s (`Enumeration` is Scala)
```scala
val nameA :: nameB :: nameC :: Nil = Enum(3)
```
* Helpful for putting human-sensical names to distinct values
* Example use cases
  * Naming states in a state machine
  * Labeling mux way selection options
  * Labeling options for interfaces
* `ChiselEnum` is a new mechanism in [experimental](https://www.chisel-lang.org/chisel3/docs/explanations/chisel-enum.html)

## Example State Machine (Raccoon)

<img src="images/raccoon.svg" alt="raccoon fsm" style="width:45%;" align="center"/>

In [ ]:
class Raccoon extends Module { // using when
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(UInt())
    })
    val hide :: wander :: rummage :: eat :: Nil = Enum(4)
    val state = RegInit(hide)
    when (state === hide) {
        when (!io.noise) { state := wander }
    } .elsewhen (state === wander) {
        when (io.noise) { state := hide }
        .elsewhen (io.trash) { state := rummage }
    } .elsewhen (state === rummage) {
        when (io.noise) { state := hide }
        .elsewhen (io.food) { state := eat }
    } .elsewhen (state === eat) {
        when (io.noise) { state := hide }
        .elsewhen (!io.food) { state := wander }
    }
    io.action := state
}
println(getVerilog(new Raccoon))

In [ ]:
class Raccoon extends Module { // using switch
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(UInt())
    })
    val hide :: wander :: rummage :: eat :: Nil = Enum(4)
    val state = RegInit(hide)
    switch(state) {
        is (hide) {
            when (!io.noise) { state := wander }
        }
        is (wander) {
            when (io.noise) { state := hide }
            .elsewhen (io.trash) { state := rummage }
        }
        is (rummage) {
            when (io.noise) { state := hide }
            .elsewhen (io.food) { state := eat }
        }
        is (eat) {
            when (io.noise) { state := hide }
            .elsewhen (!io.food) { state := wander }
        }
    }
    io.action := state
}
println(getVerilog(new Raccoon))

In [ ]:
test(new Raccoon()) { r =>
    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)

    r.io.noise.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.trash.poke(1.B)
    r.clock.step()
    r.io.action.expect(2.U)

    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(1.B)
    r.clock.step()
    r.io.action.expect(0.U)
}

In [ ]:
test(new Raccoon()) { r =>
    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(1.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(2.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)
}

## Collecting Useful Output from Simulation

* So far, we have used a PeekPoke tester to interact with the design
    * Running tests is a specific form of simulation
    * Tests can explicitly check for certain behavior
* Can set up explicit print statements (next slide)
* Can record waveforms as VCD (demo)
* Recommend building extensive tests, use prints & waveforms _only for debugging_
    * Tests should be automated (e.g. using ChiselTest), and should not require a human to inspect print output or waveforms

## Printing - In Scala (during generation)
* Can use `println`
* Scala string interpolation allows for variable names or even expressions inside strings
```scala
println(s"this is $myVal and ${foo.bar}")
```

In [ ]:
val myVal = 4
println(s"this is $myVal and it is ${myVal.getClass}")

### Printing - In Chisel (during simulation)
* Can use `printf` in C-style
```scala
printf("myVal: %d", myVal)
```
* ... or interpolation style
```scala
printf(p"myVal: $myVal")
```

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            printf("incrementing to %d\n", count)
//             printf(p"incrementing to $count\n", count)
            count := count + 1.U
        } .otherwise {
            count := 0.U
            printf("wrapping to      0\n")
        }
    }
    io.out := count
}
// println(getVerilog(new MyCounter(15)))

### Printing - In Chisel (during simulation) Demo

In [ ]:
test(new MyCounter(3)) { c =>
    c.io.en.poke(1.B)
    c.clock.step()
    c.clock.step()
    c.clock.step()
    c.clock.step()
    c.clock.step()
}

## Additional Ways to Express Literals

* Can add `.U`, `.S`, `.B` to cast to `UInt`, `SInt`, `Bool`
* Can specify bitwidth with more explicit types
* Can prefix a string with encoding `h` (hex), `o` (octal), `b` (binary)
* Can break up long literals with `_`

In [ ]:
"ha".U
"h_dead_beef".U
"ha".U
"ha".U(8.W)
"ha".asUInt(8.W)